In [1]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 656.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 4.

In [2]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [6]:
# Instalación de PyPDF2
!pip install PyPDF2

# Importar las bibliotecas necesarias
import PyPDF2


In [7]:
# Ruta local del archivo PDF
local_path = "/content/00f6e6_ManualMEGAMIG500Spanish.pdf"

# Función para extraer texto de un PDF usando UnstructuredPDFLoader y PyPDF2
def extract_text_from_pdf(pdf_path):
    loader = UnstructuredPDFLoader(file_path=pdf_path)
    pdf_obj = loader.load()

    pdf_reader = PyPDF2.PdfReader(open(pdf_path, "rb"))
    text = ''

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]  # Acceso a las paginas usando un indice
        text += page.extract_text()  # Uso del metodo extract_text()

    return text

# Función para formatear el texto en párrafos
def format_text_into_paragraphs(text):
    paragraphs = text.split('\n\n')
    formatted_text = '\n\n'.join(paragraph.strip() for paragraph in paragraphs if paragraph.strip())
    return formatted_text

# Función para guardar el texto en un archivo
def save_text_to_file(text, txt_path):
    with open(txt_path, 'w', encoding='utf-8') as file:
        file.write(text)

# Ruta de salida para el archivo .txt combinado
output_path = "/content/TextoconvertidoMEGAMIG500.txt"

# Extraer texto del PDF
pdf_text = extract_text_from_pdf(local_path)

# Formatear el texto en párrafos
formatted_text = format_text_into_paragraphs(pdf_text)

# Guardar el texto formateado en un archivo .txt
save_text_to_file(formatted_text, output_path)

print(f'Texto extraído y guardado en {output_path}')


Texto extraído y guardado en /content/TextoconvertidoMEGAMIG500.txt


In [8]:
! pip install faiss-cpu==1.7.4 mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.9 MB/s eta 0:00:00


In [9]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import requests
import numpy as np
import faiss
import os
from getpass import getpass

api_key= getpass("Type your API Key")
client = MistralClient(api_key=api_key)

Type your API Key··········


In [10]:
len(formatted_text)

33176

In [11]:
text=formatted_text

In [12]:
chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [13]:
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(
          model="mistral-embed",
          input=input
      )
    return embeddings_batch_response.data[0].embedding

In [14]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [15]:
text_embeddings.shape

(17, 1024)

In [16]:
text_embeddings

array([[-0.00710678,  0.03114319,  0.06506348, ...,  0.00994873,
         0.00481796, -0.01983643],
       [-0.01570129,  0.02578735,  0.05477905, ...,  0.01004791,
        -0.0033474 , -0.02294922],
       [-0.02400208,  0.01657104,  0.04620361, ...,  0.00724792,
        -0.00640488, -0.02763367],
       ...,
       [-0.02757263,  0.02531433,  0.05285645, ..., -0.01248932,
        -0.00832367, -0.01673889],
       [-0.03582764,  0.02583313,  0.05380249, ...,  0.00488281,
        -0.00343323, -0.00863647],
       [-0.0138092 ,  0.03259277,  0.03988647, ...,  0.01076508,
         0.0012989 , -0.00162315]])

In [17]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [18]:
question = "¿Qué se debe hacer antes de soldar?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 1024)

In [19]:
question_embeddings

array([[-0.04623413,  0.02729797,  0.05236816, ..., -0.00793457,
        -0.02848816, -0.00795746]])

In [20]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[3 9]]


In [21]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

[' ello solo conseguirá acortar la vida \nútil de los componentes y del equipo en general. \n¾ Siempre verifique que el cable de so ldar no se encuentre maltratado. \n \n \n   5Medidas de seguridad a toma r para asegurar la correct a instalación del equipo. \n \n \n \nL Asegúrese de que no exista riesgo potencial,  ta nto para el operador como para la máquina,  \nde caída de cualquier objeto extraño. \nL El polvo, ácido o material corrosivo en el ambiente del lugar de trabajo y medio ambiente \ncorrosivo no deben sobrepasar los parámetros exigidos (exceptuándose los provocados a causa de la soldadura). \nL El equipo de soldar debe instalarse protegido del sol, la lluvia, humedad excesiva o \ntemperaturas por bajo los -10° C o por sobre los  40° C. \nL Deben existir al menos 50 cms. libres alrededor del equipo de soldar  para asegurar una \nadecuada ventilación. \nL No debe introducir piezas extrañas al equipo de soldar. \nL No deben existir vibraciones excesivas en el área alrededor de

In [22]:
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""

In [23]:
def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [24]:
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Antes de soldar, se deben tomar varias medidas de seguridad y realizar un chequeo detallado. Aquí hay algunos pasos clave:

1. Verifique que no exista riesgo potencial de caída de cualquier objeto extraño, tanto para el operador como para la máquina.
2. Asegúrese de que el polvo, ácido o material corrosivo en el ambiente del lugar de trabajo y medio ambiente no sobrepasen los parámetros exigidos.
3. El equipo de soldar debe instalarse protegido del sol, la lluvia, humedad excesiva o temperaturas por debajo de los -10° C o por sobre los 40° C.
4. Deben existir al menos 50 cms. libres alrededor del equipo de soldar para asegurar una adecuada ventilación.
5. No debe introducir piezas extrañas al equipo de soldar.
6. No deben existir vibraciones excesivas en el área alrededor del lugar de trabajo de la máquina.
7. Elija un área sin interferencias electromagnéticas.
8. Asegúrese de instalar un interruptor automático de protección en la instalación eléctrica de acuerdo a las especificaciones

In [25]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
!pip install rouge-score
from rouge_score import rouge_scorer

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=0916a48059f6d71f506238f3c709863abb632253914d78cf8ece73352f070966
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [27]:
!pip install bert-score
from bert_score import score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 543.3 kB/s eta 0:00:00


In [28]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = """Antes de soldar verifique el estado de las conexiones.
Revise que la conexión de tierra del enchufe esté correcta.
Utilice ropa y herramientas apropiadas para evitar dañar la vista y la piel.
Cuando se este soldando se debe usar la máscara de soldar cubriendo toda la cabeza, sólo se puede realizar la observación visual del arco eléctrico a través del visor de la máscara.
Evite la sobrecarga de su equipo revisando previamente el ciclo de trabajo de este. Tome la precaución de que la manguera de gas nunca se encuentre presionada o doblada. Siempre verifique que el cable de soldar no se encuentre maltratado.\n
Asegúrese de que no exista riesgo potencial, tanto para el operador como para la máquina, de caída de cualquier objeto extraño.
El polvo, ácido o material corrosivo en el ambiente del lugar de trabajo y medio ambiente corrosivo no deben sobrepasar los parámetros exigidos (exceptuándose los provocados a causa de la soldadura).
El equipo de soldar debe instalarse protegido del sol, la lluvia, humedad excesiva o temperaturas por bajo los -10° C o por sobre los 40° C.
Deben existir al menos 50 cms. libres alrededor del equipo de soldar para asegurar una adecuada ventilación.
No debe introducir piezas extrañas al equipo de soldar.
No deben existir vibraciones excesivas en el área alrededor del lugar de trabajo de la máquina.
Elija un área sin interferencias electromagnéticas.
Asegúrese de instalar un interruptor automático de protección en la instalación eléctrica de acuerdo a las especificaciones indicadas por el manual de servicio del equipo.
El equipo de soldar debe instalarse horizontalmente, en una superficie plana.
Si la inclinación fuera superior a 15° se le debe adicionar elementos anti vuelco para evitar la inclinación del equipo.
No se recomienda conectar su equipo de soldar a un grupo generador.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 9.16 seconds, 0.11 sentences/sec
ROUGE-1 score: Score(precision=0.6190476190476191, recall=0.8024691358024691, fmeasure=0.6989247311827957)
ROUGE-2 score: Score(precision=0.4606205250596659, recall=0.5975232198142415, fmeasure=0.5202156334231807)
ROUGE-L score: Score(precision=0.44761904761904764, recall=0.5802469135802469, fmeasure=0.5053763440860215)
BERTScore - Precision: 0.8191155791282654, Recall: 0.8349132537841797, F1: 0.8269389271736145


In [29]:
question = "¿Qué ocurre si sucede Shock Eléctrico?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 1024)

In [30]:
D, I = index.search(question_embeddings, k=2)
print(I)

[[4 3]]


In [31]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['léctrica antes de abrir la carcaza. \n; Ante cualquier problema técnico que presente el equipo de soldar, recurra al servicio técnico \nautorizado más cercano. \n \nSe debe realizar chequeos regulares cada seis me ses por parte de personal calificado luego de la \ninstalación del equipo  de soldar. Estos chequeos deben incluir: \n \n; Limpieza de rutina para asegurar que no existan condiciones subnormales como conectores \nsueltos. \n; Inspección de los accesorios externos. \n; Chequeo del cable de soldar, para verificar que se encuentre en perfectas condiciones. \n; Reemplazo del cable alimentador si se determina que está dañado o roto. \n \nSi las normas de seguridad y de utilización no se observan atentamente, las operaciones de \nsoldadura pueden resultar peligrosas no solo pa ra el operador, sino in cluso para las personas que \nse encuentren próximas al área de trabajo.  \n \n \nPrimeros Auxilios:  \n \nEl lugar de trabajo debe contar con un botiquín de primeros auxilios equipa

In [32]:
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""

In [33]:
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Si sucede un shock eléctrico, la persona accidentada puede quedar inconsciente. En tal caso, es importante no tocarla si todavía está en contacto con algún cuerpo posiblemente energizado. Lo primero que se debe hacer es cortar el suministro eléctrico que alimenta el equipo y luego recurrir a los cuidados de primeros auxilios. Para alejar los cables y/o partes energizadas de la víctima, se recomienda utilizar trozos de madera bien seca, como una escoba o cualquier otro material aislante. El lugar de trabajo debe contar con un botiquín de primeros auxilios equipado para socorrer a posibles víctimas de un shock eléctrico y tratar posibles quemaduras a la piel y/o ojos causadas por la exposición directa a la luz y calor emitidos en el proceso de soldadura. El shock eléctrico puede ser mortal.


In [34]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = """Si la persona accidentada esta inconsciente y se sospecha un shock eléctrico, tenga la precaución de
no tocarla si ha quedado en contacto con algún cuerpo posiblemente energizado.
Corte el suministro eléctrico que alimenta el equipo y recurra a los cuidados de primeros auxilios.
Para alejar los cables y/o partes energizadas de la víctima, se recomienda utilizar trozos de madera
bien seca, como una escoba o cualquier otro material aislante.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.73 seconds, 0.58 sentences/sec
ROUGE-1 score: Score(precision=0.4755244755244755, recall=0.8947368421052632, fmeasure=0.6210045662100456)
ROUGE-2 score: Score(precision=0.39436619718309857, recall=0.7466666666666667, fmeasure=0.5161290322580645)
ROUGE-L score: Score(precision=0.4195804195804196, recall=0.7894736842105263, fmeasure=0.547945205479452)
BERTScore - Precision: 0.8026891350746155, Recall: 0.8870231509208679, F1: 0.8427515625953674


In [36]:
question = "¿Cuáles son los requerimientos para la fuente de poder?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Los requerimientos para la fuente de poder según la información proporcionada son:

a) El oscilograma de voltaje debe mostrar una onda de seno, y la oscilación de la frecuencia no debe exceder ± 1% del valor indicado.

b) La oscilación del voltaje no debe superar ± 10% del valor indicado.

c) El desbalance de la alimentación trifásica no debe exceder el 5%.

Además, los datos técnicos para el equipo de soldar MEGAMIG 500 mencionan que el voltaje de entrada es de 3PH – 380 V, la frecuencia es de 50 Hz, y la corriente de entrada es de 40 A. La potencia absorbida de entrada es de 30 KVA.


In [37]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = """El oscilograma de voltaje debe mostrar una onda de seno, la oscilación de la frecuencia no
debe exceder ± 1% del valor indicado. La oscilación del voltaje no debe superar ± 10 % del valor indicado.
El desbalance de la alimentación trifásica no debe exceder el 5%.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.55 seconds, 0.64 sentences/sec
ROUGE-1 score: Score(precision=0.42105263157894735, recall=1.0, fmeasure=0.5925925925925926)
ROUGE-2 score: Score(precision=0.3893805309734513, recall=0.9361702127659575, fmeasure=0.55)
ROUGE-L score: Score(precision=0.42105263157894735, recall=1.0, fmeasure=0.5925925925925926)
BERTScore - Precision: 0.7853518724441528, Recall: 0.9210900664329529, F1: 0.847822368144989


In [38]:
question = "¿Cuáles son las principales tareas de mantenimiento?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Las principales tareas de mantenimiento para el equipo de soldar incluyen:

1. Remoción de Polvo: Asegúrate de que la máquina no esté conectada a una fuente de poder antes de proceder a la remoción de polvo.

2. Almacenamiento seguro: Si vas a dejar tu equipo por un largo periodo de inactividad, hazlo en un ambiente seco, limpio y con buena ventilación.

3. Limpieza periódica: A través de personal especializado, se debe realizar limpieza por medio de aire comprimido seco para limpiar el interior del equipo de soldar. También se debe revisar si existen componentes o cables sueltos, y proceder a la reparación o reemplazo correspondiente.

4. Mantenimiento del cable de poder: Es necesario chequear periódicamente el cable de alimentación del equipo de soldar.

5. Revisión de Cables de soldar: Al momento de utilizar el equipo de soldar, el operador debe revisar que los cables de soldar no estén desconectados y no presenten cortes en el aislamiento.

6. Revisión del circuito de gas: Revise r

In [39]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = """Remoción de Polvo .Debe asegurarse que la máquina no esté conectada a una fuente de
poder antes de proceder a la remoción de polvo.
Asegurece que al dejar su equipo por un largo periodo de inactividad, lo haga en un ambiente seco, limpio y con buena ventilación.
Limpieza periódica. A través de personal especializado, se debe realizar limpieza por medio de aire comprimido seco para limpiar el interior del equipo de soldar. También se
debe revisar si existen componentes o cables sueltos, si los hubiera, proceda a la reparación o reemplazo correspondiente. Generalmente cuando no existe polvo en demasía la limpieza
se debe realizar periódicamente, y cuando existe mucha acumulación de polvo la limpieza se debe realizar con mayor frecuencia.Mantenga el cable de poder en buen estado Es necesario chequear periódicamente el cable
de alimentación del equipo de soldar, sin embargo, es aconsejable revisar cada vez que la máquina sea utilizada de forma portátil.
Revise Cables de soldar. Al momento de utilizar el equipo de soldar, el operador debe revisar que los cables de soldar no estén desconectados, no presenten cortes en el
aislamiento, etc. En caso de que se presente alguno de estos problemas se debe proceder al cambio o reparación correspondiente.
Revise regularmente que el circuito de gas no presente fugas y se mantenga completamente sellado. Verifique ademas que el ventilador y el motor alimentador estén funcionando
correctamente sin sonidos anormales ni cables sueltos. Limpie periódicamente la boquilla de escoria y suciedad.
Utilice alambre Mig de calidad, que cuente con certificaciones. Reemplace los rodillos si se encuentran gastados o deteriorados. Apriete los rodillos contra
el alambre sin sobrepresionar para asegurar un buen desplazamiento de éste.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 8.75 seconds, 0.11 sentences/sec
ROUGE-1 score: Score(precision=0.7874015748031497, recall=0.6734006734006734, fmeasure=0.7259528130671508)
ROUGE-2 score: Score(precision=0.6521739130434783, recall=0.5574324324324325, fmeasure=0.6010928961748634)
ROUGE-L score: Score(precision=0.7165354330708661, recall=0.6127946127946128, fmeasure=0.6606170598911071)
BERTScore - Precision: 0.7953457832336426, Recall: 0.7899434566497803, F1: 0.792635440826416


In [40]:
question = "¿Antes de armar o desarmar la pistola que se debe hacer?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Antes de armar o desarmar la pistola, se debe desconectar el equipo de soldar de la fuente de energía eléctrica. Esto es para garantizar la seguridad del operador y evitar daños a los componentes y al equipo en general.


In [41]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response = "Antes de armar o desarmar la pistola se la debe desconectar de la energía eléctrica." # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.73 seconds, 1.37 sentences/sec
ROUGE-1 score: Score(precision=0.40476190476190477, recall=1.0, fmeasure=0.576271186440678)
ROUGE-2 score: Score(precision=0.2926829268292683, recall=0.75, fmeasure=0.4210526315789473)
ROUGE-L score: Score(precision=0.38095238095238093, recall=0.9411764705882353, fmeasure=0.5423728813559321)
BERTScore - Precision: 0.7762361764907837, Recall: 0.923952043056488, F1: 0.8436772227287292


In [42]:
question = "¿Cuándo pueden resultar peligrosas las operaciones de soldadura?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Las operaciones de soldadura pueden resultar peligrosas en varias situaciones según la información proporcionada:

1. Cuando no se verifica el estado de las conexiones y la conexión de tierra del enchufe no está correcta.
2. Cuando no se trabaja en lugares bien ventilados o con extracción forzada, lo que puede causar la acumulación de humos y gases dañinos para la salud.
3. Cuando no se protege la zona de larco eléctrico mediante el uso de cortinas protectoras adecuadas, lo que puede afectar la vista de las personas cercanas.
4. Cuando personal no capacitado intenta mover o realizar ajustes al equipo de soldar.
5. Cuando personas con marcapasos cardíacos u otros artefactos sensibles se acercan al equipo de soldar en plena operación, debido a la inducción electromagnética producida por el equipo.
6. Cuando se utiliza el equipo de soldar para descongelar tuberías o sin los paneles, lo que puede ser peligroso para el operador y dañar seriamente el funcionamiento del equipo.
7. Cuando se s

In [43]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""Si las normas de seguridad y de utilización no se observan atentamente, las operaciones de
soldadura pueden resultar peligrosas no solo para el operador, sino incluso para las personas que
se encuentren próximas al área de trabajo.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.18 seconds, 0.31 sentences/sec
ROUGE-1 score: Score(precision=0.10344827586206896, recall=0.6923076923076923, fmeasure=0.18)
ROUGE-2 score: Score(precision=0.05, recall=0.34210526315789475, fmeasure=0.08724832214765102)
ROUGE-L score: Score(precision=0.06130268199233716, recall=0.41025641025641024, fmeasure=0.10666666666666666)
BERTScore - Precision: 0.6355978846549988, Recall: 0.7504872679710388, F1: 0.6882811188697815


In [44]:
question = "¿Cómo influye la velocidad del alambre?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

La velocidad del alambre en el proceso de soldadura es un factor crucial que afecta directamente la calidad de la soldadura. De acuerdo con la información proporcionada, ajustar la velocidad del alambre permite controlar la corriente de soldadura. Un ajuste continuo de la velocidad del alambre puede used para ajustar la corriente de soldadura y también para establecer el tiempo de Burn Back. Por lo tanto, la velocidad del alambre tiene un impacto significativo en la calidad y eficiencia del proceso de soldadura.


In [45]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""La velocidad del alambre influye directamente en la corriente de soldadura. A un
mismo voltaje, a mayor velocidad del alambre mayor es la corriente de soldadura.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.79 seconds, 1.27 sentences/sec
ROUGE-1 score: Score(precision=0.2235294117647059, recall=0.7307692307692307, fmeasure=0.34234234234234234)
ROUGE-2 score: Score(precision=0.14285714285714285, recall=0.48, fmeasure=0.2201834862385321)
ROUGE-L score: Score(precision=0.2, recall=0.6538461538461539, fmeasure=0.3063063063063063)
BERTScore - Precision: 0.7516089081764221, Recall: 0.7950412034988403, F1: 0.7727152109146118


In [46]:
question = "¿Cómo se debe conectar el equipo de soldar y el sistema de gas?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Para conectar el equipo de soldar y el sistema de gas, primero debes asegurarte de que el regulador de gas esté firmemente atornillado a su conexión para evitar fugas de gas. Luego, conecta y fija firmemente el cable de calefacción a la conexión del calefactor del panel trasero de la fuente de poder.


In [47]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""Asegúrese que el regulador de gas este firmemente atornillado a su conexión para evitar fugas de
gas. Luego conecte y fije firmemente el cable de calefacción a la conexión del calefactor del panel trasero
de la fuente de poder.
Conecte un extremo de la manguera de gas a la salida del regulador de presión y el otro extremo a la
entrada de gas del alimentador de alambre. """ # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.68 seconds, 1.46 sentences/sec
ROUGE-1 score: Score(precision=0.8035714285714286, recall=0.625, fmeasure=0.703125)
ROUGE-2 score: Score(precision=0.6909090909090909, recall=0.5352112676056338, fmeasure=0.6031746031746031)
ROUGE-L score: Score(precision=0.6964285714285714, recall=0.5416666666666666, fmeasure=0.6093749999999999)
BERTScore - Precision: 0.872178852558136, Recall: 0.8601152896881104, F1: 0.8661050796508789


In [48]:
question = "¿Cómo se debe realizar la instalacion y conexión del alimentador de alambre SUPER TRACK?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Para realizar la instalación y conexión del alimentador de alambre SUPER TRACK, siga los siguientes pasos:

1. Conecte un extremo de la manguera de gas a la salida del regulador de presión y el otro extremo a la entrada de gas del alimentador de alambre.

2. Seleccione el diámetro del alambre según el trabajo a ejecutar. En función de este diámetro, elija el carrete, flexible y boquilla adecuados.

3. Abra la cubierta del alimentador de alambre SUPER TRACK e instale el carrete en el eje del carrete.

4. Introduzca el alambre en el tubo de entrada del mecanismo de arrastre y alínielo en la ranura del rodillo de empuje adecuado.

5. Vuelva a centrarlo en la boquilla de salida y active el mecanismo de presión.

6. Conecte la pistola en el conector correspondiente del panel frontal y apriete firmemente.

7. Conecte un extremo del cable de tierra al conector correspondiente en el panel frontal de la fuente de poder y atornille firmemente. Con el otro extremo, aprisione firmemente la pieza a

In [49]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""Seleccione el diámetro del alambre según el trabajo a ejecutar. En función de este diámetro elija el
carrete, flexible y boquilla adecuados.
Abra la cubierta del alimentador de alambre SUPER TRACK e instale en el eje del carrete.
Introduzca el alambre en el tubo de entrada del mecanismo de arrastre y alínielo en la ranura del
rodillo de empuje adecuado. Vuelva a centrarlo en la boquilla de salidad y active el mecanismo de
presión. """ # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.43 seconds, 0.70 sentences/sec
ROUGE-1 score: Score(precision=0.43646408839779005, recall=0.9875, fmeasure=0.6053639846743295)
ROUGE-2 score: Score(precision=0.40555555555555556, recall=0.9240506329113924, fmeasure=0.5637065637065638)
ROUGE-L score: Score(precision=0.43646408839779005, recall=0.9875, fmeasure=0.6053639846743295)
BERTScore - Precision: 0.786047101020813, Recall: 0.8880327939987183, F1: 0.833933413028717


In [50]:
question = "¿Qué sucede si el equipo de soldar es operado por largos períodos de tiempo?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = f"""
La información de contexto está a continuación..
---------------------
{retrieved_chunk}
---------------------
Dada la información del contexto y no el conocimiento previo, responda la consulta.
Consulta: {question}
Respuesta:
"""
# Ejecuta la función run_mistral con el prompt generado
response = run_mistral(prompt)

# La respuesta generada por el modelo se almacena en la variable 'response'
print(response)

Si el equipo de soldar es operado por largos períodos de tiempo, se activará el Led indicador de protección activada (termostato, N° 1) en el panel. Esto significa que la temperatura interna del equipo de soldar ha superado la temperatura de su diseño. En este caso, el equipo de soldar se detendrá y el ventilador seguirá funcionando. Una vez que la temperatura baje lo suficiente, el led indicador de protección activada se apagará y el equipo de soldar quedará en condiciones de reiniciar su operación.


In [51]:
# para comparar
generated_response = str(response)  # La respuesta generada por el modelo
# La respuesta textual del manual técnico
reference_response ="""Cuando el equipo de soldar es operado por largos períodos de tiempo, en el panel se activará
el Led indicador de protección activada (termostato, N° 1) lo que significa que la
temperatura interna del equipo de soldar ha sobrepasado la temperatura de su diseño. En
este caso el equipo de soldar se detendrá y el ventilador seguirá funcionando. Una vez que
la temperatura baje lo suficiente, el led indicador de protección activada se apagará y el
equipo de soldar quedará en condiciones de reiniciar su operación.""" # La respuesta esperada

# Calcular ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(reference_response, generated_response)

# Calcular BERTScore
P, R, F1 = score([generated_response], [reference_response], lang="es", verbose=True)

# Imprime resultados
print(f"ROUGE-1 score: {rouge_scores['rouge1']}")
print(f"ROUGE-2 score: {rouge_scores['rouge2']}")
print(f"ROUGE-L score: {rouge_scores['rougeL']}")
print(f"BERTScore - Precision: {P.mean().item()}, Recall: {R.mean().item()}, F1: {F1.mean().item()}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.29 seconds, 0.78 sentences/sec
ROUGE-1 score: Score(precision=0.9666666666666667, recall=0.9560439560439561, fmeasure=0.9613259668508287)
ROUGE-2 score: Score(precision=0.9213483146067416, recall=0.9111111111111111, fmeasure=0.9162011173184358)
ROUGE-L score: Score(precision=0.9333333333333333, recall=0.9230769230769231, fmeasure=0.9281767955801106)
BERTScore - Precision: 0.9813107848167419, Recall: 0.9798988103866577, F1: 0.980604350566864
